# Autograd

## 0. torch.autograd.grad()
새롭게 정의된 텐서는 기본적으로 requires_grad가 False입니다.

In [1]:
import numpy as np
import torch

a = torch.tensor(np.arange(5.))
print(a.requires_grad)

False


이렇게 정의된 텐서는 (시간과 메모리를 위해) gradient tracking을 하지 않습니다.

In [2]:
# g = torch.autograd.grad(a.mean(), a)
# RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

그리고 이는 다음과 같이 변경할 수 있습니다.

In [3]:
a = torch.tensor(np.arange(5.))
a.requires_grad_() # a.requires_grad_(True) 도 됩니다.
g = torch.autograd.grad(a.mean(), a)
print(g)
print(a.grad)

(tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], dtype=torch.float64),)
None


여기에서 기억하면 좋은 점은
* autograd.grad 함수의 return값은 항상 튜플을 가진다는 것과
* autograd.grad 함수는 backward 함수와 다르게 leaf node 외의 tensor의 grad 값을 계산할 수 있다는 것과
* autograd.grad 함수는 backward 함수와 다르게 미분한 변수(a)의 grad 값을 바꾸지 않는다는 점입니다.

따라서 autograd.grad 함수는 원하는 부분의 미분을 구할 때 용이합니다. (e.g. [WGAN-GP](https://github.com/sungyubkim/GP-Works/blob/master/Gradient_Penalty_Works.ipynb), [MAML](https://github.com/sungyubkim/maml-pytorch/blob/master/cls-mimgnet/main.py))

## 1. second order gradient

pytorch에서는 Jacobian matrix를 효율적으로 구하는 것이 [어렵습니다](https://discuss.pytorch.org/t/how-to-compute-jacobian-matrix-in-pytorch/14968).(혹시 쉽게 구하는 방법을 알게되면 제게 알려주세요!) 

그 가장 큰 이유 중 하나는 backward 함수와 grad 함수는 scalar에 대해서밖에 미분을 하지 못하기 때문입니다.

In [4]:
a = torch.tensor(np.arange(5.)).requires_grad_()
b = a + 10

# b.backward()
# RuntimeError: grad can be implicitly created only for scalar outputs

# grad = torch.autograd.grad(b, a)
# RuntimeError: grad can be implicitly created only for scalar outputs

따라서 직접 Jacobian을 구하는 것은 어렵지만, 어떤 scalar가 gradient로 정의되었다면 이를 미분하는 것(즉, 원래 변수에 대해 Hessian vecotr product를 하는 것)은 다음과 같이 할 수 있습니다.

In [5]:
a = torch.tensor(np.arange(5.)).requires_grad_()
# b = a + 10 # 주석을 풀고 실행시켜보세요. 왜 안될까요?
b = a * a
g = torch.autograd.grad(b.mean(), a, create_graph=True)[0]
c = g.norm(p=2, dim=0)
c.backward()
print(a.grad)

tensor([0.0000, 0.0730, 0.1461, 0.2191, 0.2921], dtype=torch.float64)


## 2. torch.no_grad()
일반적인 텐서와 다르게 Parameter들은 (거의 항상) gradient tracking을 합니다.

In [6]:
import torch.nn as nn

a = nn.Parameter(torch.ones(5))
print(a.requires_grad)

True


하지만 validation과 test 시에는 이러한 gradient tracking이 필요 없습니다. 이러한 경우에는 no_grad context를 사용해 메모리와 시간을 절약할 필요가 있습니다.

In [7]:
with torch.no_grad():
    b = 2 * a.mean()
    
# b.backward()
# RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

특히 no_grad context를 사용하지 않은 경우에는 training과 valid, test의 메모리를 따로 할당하는 경우가 있으므로 항상 확인을 하면서 실행시키는 것이 좋습니다.

만약 no_grad context 안에서 부분적으로 gradient tracking이 필요한 경우에는 다음과 같이 활성화할 수 있습니다.

In [8]:
with torch.no_grad():
    with torch.enable_grad():
        b = 2 * a.mean()
        
b.backward()
print(a.grad)

tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000])


## 3. parameters() and named_parameters()

nn.Module 객체 메소드인 parameters()는 다음과 같이 iterator(generator)를 리턴합니다.

In [9]:
class MLP(nn.Module):
    def __init__(self, input_dim=10, hidden_dim=100, output_dim=1):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.layers = nn.Sequential(
        nn.Linear(self.input_dim, self.hidden_dim),
        nn.ReLU(True),
        nn.Linear(self.hidden_dim, self.output_dim)
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x

mlp = MLP()
print(mlp.parameters())
for param in mlp.parameters():
    print(param.shape)

<generator object Module.parameters at 0x7f0bdca3fcf0>
torch.Size([100, 10])
torch.Size([100])
torch.Size([1, 100])
torch.Size([1])


이를 기반으로 다음과 같이 직접 gradient descent를 할 수도 있습니다.

In [10]:
x = torch.zeros(10)
y = mlp(x)
grad = torch.autograd.grad(y.mean(), mlp.parameters())

for p, g in zip(mlp.parameters(), grad):
    print('Before : ',p.abs().mean())
    p = p - g
    print('After : ',p.abs().mean())

Before :  tensor(0.1589, grad_fn=<MeanBackward0>)
After :  tensor(0.1589, grad_fn=<MeanBackward0>)
Before :  tensor(0.1397, grad_fn=<MeanBackward0>)
After :  tensor(0.1416, grad_fn=<MeanBackward0>)
Before :  tensor(0.0528, grad_fn=<MeanBackward0>)
After :  tensor(0.0982, grad_fn=<MeanBackward0>)
Before :  tensor(0.0563, grad_fn=<MeanBackward0>)
After :  tensor(1.0563, grad_fn=<MeanBackward0>)


한편 paramters는 각 parameter들이 어떤 layer인지 알 수 없기 떄문에 이를 다음과 같이 named_parameters 함수를 사용해 변형할 수도 있습니다.

이때 named parameters의 리턴값은 [OrderedDictionary](https://docs.python.org/3/library/collections.html) 처럼 사용할 수 있습니다.

In [11]:
for (k,v), g in zip(mlp.named_parameters(), grad):
    print('{} Before : '.format(k),v.abs().mean())
    v = v - g
    print('{} After : '.format(k), v.abs().mean())

layers.0.weight Before :  tensor(0.1589, grad_fn=<MeanBackward0>)
layers.0.weight After :  tensor(0.1589, grad_fn=<MeanBackward0>)
layers.0.bias Before :  tensor(0.1397, grad_fn=<MeanBackward0>)
layers.0.bias After :  tensor(0.1416, grad_fn=<MeanBackward0>)
layers.2.weight Before :  tensor(0.0528, grad_fn=<MeanBackward0>)
layers.2.weight After :  tensor(0.0982, grad_fn=<MeanBackward0>)
layers.2.bias Before :  tensor(0.0563, grad_fn=<MeanBackward0>)
layers.2.bias After :  tensor(1.0563, grad_fn=<MeanBackward0>)


여기서 한가지 함정은 보시다시피 앞에서 한번 gradient descent를 했음에도 두번째에 그대로 파라미터 값들이 유지가 된다는 점입니다. 이를 방지하기 위해서는 다음과 같이 Parameter가 아닌 data속성을 바꿔줘야 합니다. 


이는 일반적으로 연산을 이용해 기존 tensor의 python variable에 새로운 tensor를 assign하면 computational graph에서 python variable을 새로운 tensor로 인식하기 때문입니다.

In [12]:
for (k,v), g in zip(mlp.named_parameters(), grad):
    print('{} Before : '.format(k),v.abs().mean())
    v.data =  v - g
    print('{} After : '.format(k), v.abs().mean())
    
for (k,v), g in zip(mlp.named_parameters(), grad):
    print('{} Before : '.format(k),v.abs().mean())
    v.data = v - g
    print('{} After : '.format(k), v.abs().mean())

layers.0.weight Before :  tensor(0.1589, grad_fn=<MeanBackward0>)
layers.0.weight After :  tensor(0.1589, grad_fn=<MeanBackward0>)
layers.0.bias Before :  tensor(0.1397, grad_fn=<MeanBackward0>)
layers.0.bias After :  tensor(0.1416, grad_fn=<MeanBackward0>)
layers.2.weight Before :  tensor(0.0528, grad_fn=<MeanBackward0>)
layers.2.weight After :  tensor(0.0982, grad_fn=<MeanBackward0>)
layers.2.bias Before :  tensor(0.0563, grad_fn=<MeanBackward0>)
layers.2.bias After :  tensor(1.0563, grad_fn=<MeanBackward0>)
layers.0.weight Before :  tensor(0.1589, grad_fn=<MeanBackward0>)
layers.0.weight After :  tensor(0.1589, grad_fn=<MeanBackward0>)
layers.0.bias Before :  tensor(0.1416, grad_fn=<MeanBackward0>)
layers.0.bias After :  tensor(0.1537, grad_fn=<MeanBackward0>)
layers.2.weight Before :  tensor(0.0982, grad_fn=<MeanBackward0>)
layers.2.weight After :  tensor(0.1706, grad_fn=<MeanBackward0>)
layers.2.bias Before :  tensor(1.0563, grad_fn=<MeanBackward0>)
layers.2.bias After :  tensor(2

## 4. clone() and detach()

Python variable에 tensor를 할당하게 되면 이는 동일한 tensor로 인식합니다.

In [13]:
a = torch.tensor(np.arange(5.)).requires_grad_()
b = a
c = b + 10
c.mean().backward()
print(a.grad)
print(b.grad)

tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], dtype=torch.float64)
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], dtype=torch.float64)


그에 비해 clone 함수는 leaf node(a)를 새롭게 정의한 tensor(b)와 다른 tensor로 인식하게 합니다.

In [14]:
a = torch.tensor(np.arange(5.)).requires_grad_()
b = a.clone() # clone function makes b non-leaf node
c = b + 10
c.mean().backward()
print(a.grad)
print(b.grad)

tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], dtype=torch.float64)
None


마지막으로 detach 함수는 a와 b 사이의 gradient tracking을 단절합니다.

In [15]:
a = torch.tensor(np.arange(5.)).requires_grad_()
b = a.detach().requires_grad_()
c = b + 10
c.mean().backward()
print(a.grad)
print(b.grad)

None
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], dtype=torch.float64)


## 5. gradient clipping 

계산된 gradient는 때때로 clipping이 필요할 때가 있습니다.(e.g. Reinforcement Learning, Generative Adversarial Learning)

이럴때는 

* torch.nn.utils.clip_grad_norm_()
* torch.nn.utils.clip_grad_value_()

를 사용하면 됩니다.

In [16]:
x = torch.zeros(10)
y = mlp(x)
y.mean().backward()

torch.nn.utils.clip_grad_value_(mlp.parameters(), 1)

for (k,v), g in zip(mlp.named_parameters(), grad):
    print('{} th grad : '.format(k), g.abs().max())

layers.0.weight th grad :  tensor(0.)
layers.0.bias th grad :  tensor(0.0998)
layers.2.weight th grad :  tensor(0.3149)
layers.2.bias th grad :  tensor(1.)


In [17]:
x = torch.zeros(10)
y = mlp(x)
y.mean().backward()

print('Total norm of parameters are ',torch.nn.utils.clip_grad_norm_(mlp.parameters(), 1))

Total norm of parameters are  5.997436233739499
